In [ ]:
import numpy as np
import matplotlib.pyplot as pl
from numpy import linalg as LA
from operator import itemgetter
from sklearn import linear_model,datasets
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest,f_classif

In [ ]:
train_data = np.loadtxt("/Users/qinghongxu/Documents/MATH895/HW2/spam.data.txt",
                        delimiter=' ')

In [ ]:
from sklearn.model_selection import KFold # import KFold
X = np.array(train_data[:,0:56]) # create an array
y = np.array(train_data[:,57]) # Create another array
kf = KFold(n_splits=10, random_state=None, shuffle=True) 
# Define the split - into 10 folds 

In [ ]:
# Linear Regrssion
residual_list = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm = linear_model.LinearRegression()
    lm.fit(X_train, y_train)
    predictions = lm.predict(X_test)
    residual = float(1)/ y_test.shape[0] * np.sum(np.power(predictions - y_test,2))
    residual_list = np.append(residual,residual_list)
accuracy = float(1)/10*np.sum(residual_list) #CV error
std = np.sqrt(float(1)/9*np.var(residual_list - accuracy)) #Std error
#print(accuracy)
#print(std)

In [ ]:
# Ridge regression
accuracy_all_rr = []
std_all_rr = []
coerr = np.linspace(0,10,15)
for alpha in coerr:
    residual_list = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        reg = linear_model.Ridge(alpha)
        reg.fit(X_train, y_train)
        predictions = reg.predict(X_test)
        residual = float(1)/ y_test.shape[0] * np.sum(np.power(predictions - y_test,2))
        residual_list = np.append(residual,residual_list)
    accuracy = float(1)/10 * np.sum(residual_list) #CV error
    accuracy_all_rr = np.append(accuracy_all_rr,accuracy)
    #std = np.sqrt(float(1)/9* np.sum(np.power(residual_list-accuracy,2)))
    std = np.sqrt(float(1)/9* np.var(residual_list)) #Std error
    std_all_rr = np.append(std_all_rr,std)
print(accuracy_all_rr)
print(std_all_rr)

In [ ]:
# Lasso 
accuracy_all_lr = []
std_all_lr = []
coelr = np.linspace(0.0001,0.1,10)
for alpha in coelr:
    residual_list = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        reg = linear_model.Lasso(alpha)
        reg.fit(X_train, y_train)
        predictions = reg.predict(X_test)
        residual = float(1)/ y_test.shape[0] * np.sum(np.power(predictions - y_test,2))
        residual_list = np.append(residual,residual_list)
    accuracy = float(1)/10 * np.sum(residual_list) #CV error
    accuracy_all_lr = np.append(accuracy_all_lr,accuracy)
    #std = np.sqrt(float(1)/9* np.sum(np.power(residual_list-accuracy,2)))
    std = np.sqrt(float(1)/9* np.var(residual_list)) #Std error
    std_all_lr = np.append(std_all_lr,std)
print(accuracy_all_lr)
print(std_all_lr)

In [ ]:
# Forward stepwise selection
accuracy_all_fs = []
std_all_fs = []
subset = [1,5,10,15,20,25,30,35,40,45,50]
for k in subset:
    residual_list = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]       
        index = SelectKBest(f_classif, k).fit(X_train, y_train).get_support(indices=True)
        X_train_new = X_train[:,index]    
        X_test_new = X_test[:,index]    
        lm = linear_model.LinearRegression()
        lm.fit(X_train_new, y_train)
        predictions = lm.predict(X_test_new)        
        residual = float(1)/ y_test.shape[0] * np.sum(np.power(predictions
                                                               - y_test,2))
        residual_list = np.append(residual,residual_list)
    accuracy = float(1)/10 * np.sum(residual_list) #CV error
    accuracy_all_fs = np.append(accuracy_all_fs,accuracy)
    #std = np.sqrt(float(1)/9* np.sum(np.power(residual_list-accuracy,2)))
    std = np.sqrt(float(1)/9* np.var(residual_list)) #Std error
    std_all_fs = np.append(std_all_fs,std)
print(accuracy_all_fs)
print(std_all_fs)

In [ ]:
pl.errorbar(coerr, accuracy_all_rr, std_all_rr, fmt='o')
pl.errorbar(10, accuracy,std, fmt='+')
pl.xlabel('alpha')
pl.ylabel('CV error')
pl.title('Linear regression and ridge regression')
pl.legend(["CV/Std error for ridge regression",
           "CV/Std error for linear regression"],
          loc="upper left", bbox_to_anchor=(0, -0.2))
pl.grid()
pl.show()

In [ ]:
a = np.argmin(accuracy_all_fs)
y = accuracy_all_fs[a] + std_all_fs[a]
b = [0] * len(subset)
y = b + y
pl.plot(subset,y,'--')
xx = [30] * 2
yy = np.linspace(0.1,0.22,2)
pl.plot(xx,yy,'--')
pl.errorbar(subset, accuracy_all_fs, std_all_fs, fmt='-o')
pl.xlabel('subset k')
pl.ylabel('CV error')
pl.title('Forward stepwise selection')
pl.grid()
pl.show()

In [ ]:
pl.errorbar(coelr, accuracy_all_lr, std_all_lr, fmt='o')
pl.xlabel('alpha')
pl.ylabel('CV error')
pl.title('Lasso')
pl.legend(["CV/Std error for Lasso"]
          , loc="upper left", bbox_to_anchor=(0, -0.2))
pl.grid()
pl.show()